In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Heart Failure Prediction - Project

1. Let's doooo this. This is my first official data science mini weekend project attempt. It is supposed to be a fun learning process. Let's see how capable this human is. 

### Library Importing

In [ ]:
from math import sqrt
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

#from sklearn.preprocessing import CategoricalEncoder as ce
#!pip install --upgrade category_encoders
#from feature_engine import categorical_encoders as ce
#from feature_engine import missing_data_imputers as mdi

## Step 1: Data Import


In [ ]:
heart_failure_data_file_path = '../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv'
heart_failure_data = pd.read_csv(heart_failure_data_file_path) 

## Step 2: Observe and Understand Data

So apparently this data has no mismatched or missing data (according to the page where I found this data)

Quick real world breakdown and understanding of data. (might not be 100% accurate as i might have miss interpreted what the Dr. who explained this stuff said to me)

This task is not about the english. So don't get bothered by language errors. :)

* **Age:** obviously older you are more likely you are of heart failure. longer exposure to bad habits, body start breaking dwon naturally etc.

* **Anaemia:** heart failure is basically defined to occur when your heart/heart pump cannot support your body. So if you have anaemia you carry less oxygen, now if you superimpose that with another problem then you reach heart failure faster. So anaemia is like a booster towards heart failure. It worsens the heart problem. 

* **creatinine_phosphokinase:**The CPK enzyme is released when cardiac muscle gets destroyed/damaged. So if you have heart failure the CPK enzyme count should be more. Testing for this is tricky though because apparently they use molecular mass to determine what enzyme it is. This allows for error. So just because you have a high count doesnt mean you necesarily have heart failure. This test is still more accurate than the serum_creatinine test.

* **diabetes:**

* **ejection fraction:** under 40% means you are having a systolic failure. ejection fraction is the ratio of blood into and out of your heart. Obviously the smaller the ratio the less blood your body pumps out. This is a problem ofcoarse. (There is also preserved ejection ratio which will be discussed in high blood pressure) - YOU CAN STILL BE HAVING A HEART FAILURE EVEN IF THIS IS NORMAL. if its not normal you are definitely having a heart issue. 

* **high blood pressure:** This is a form of preserved ejection ratio or diastolic failure. basically something happens where your muscle wall thickens on the one side. This means less volume of blood can go into your heart. So although the ratio of blood in and out stays the same, the volume in and out decreases.

* **Platelets:** more plateletes more likely you are to clot, especially when youre older. e.g your valves dont work as well, so there is turbulance which leads to clotting of plateletes. 

* **serum_creatinine:** enzyme that is released when there is a damage in muscle tissue. (not specific to heart). This test is more likely for error. Could indicate heart failure but doesnt necessarily do so. If the person consumes creatinine supplements it affect the test results.

* **serum_sodium:** When you have allot of sodium in your blood it coulf be an indication of low blood pressure. When you have low blood pressure your bodu tries to fix that and it absorbs water and sodium into your blood. 

* **sex:** not a medical reasoning thats obvious

* **smoking:**

* **time:**

* **death event:** are these just general death events or death events due to heart failure. I am going to make a mega assumption that these are indeed deaths due to heart failure otherwise I don't think I am on a level that can solve this problem otherwise. 


Okay so this is all the theoretical trend and knowledge data. But there is no reason why there isnt a correlation that exists that has not been understood yet. This is where Machine Learning helps. But for my starting point I am going to try create a ML model that works with the knwoledge I have.

In [ ]:
heart_failure_data.describe()


## Step 3:Organise Data
credit needs to go where its due: I worked through  [this](https://towardsdatascience.com/binary-classification-example-4190fcfe4a3c)  blog as guidance.


Checking correlation - high correkation is not ideal

In [ ]:
#cheching correlarion between our features ---- im not sure how its checking --- but it seems as if our features hoe very low correlations.
#Our highest correlation is deat_event and time.
#it seems that the whole point of checking for correlations is so that if there is a high correlation you can actually remove those features. 
heart_failure_data_copy = heart_failure_data.copy()
corr_matrix = heart_failure_data_copy.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper




Checking for missing values

In [ ]:

#ALthough we were told there are no missing values we can still check it incase
for var in heart_failure_data.columns:
    print(var, 'percent of missing values', heart_failure_data[var].isnull().mean().round(3))


unique stuff

The data seems to be unique across the features minus ofcoarse the boolean styled data

In [ ]:

#unique values
print('The unique values')
for var in heart_failure_data.columns:
    print(var, heart_failure_data[var].unique(), '\n')
 

variability is good, a lack of variability means the feature isnt as useful

In [ ]:
   
#variability test
#there is no feature that has a category that holds the majority (90%), so all of the features have enough variability to keep them
for var in heart_failure_data.columns:
    print(var,'\n', heart_failure_data[var].value_counts()/len(heart_failure_data))
    
# So it looks as if we did not need to delete any features from these tests

Cardinality of Categorical and Discrete Features

In [ ]:
heart_failure_features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']
continuous_features = ['age', 'time']
discrete_features = ['creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

for var in heart_failure_features :
    print(var, 'has', heart_failure_data[var].nunique(), 'unique categories')

Distributions and Outliers

There is allot to talk about and understand here, basically if your data has a skewed distribution you can fix it with different transformations. But how does that help? ... need to explore that idea further to properly understand it

In [ ]:
for var in continuous_features:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    fig = heart_failure_data.boxplot(column=var)
    fig.set_title('outliers')
    fig.set_ylabel(var)
    
    plt.subplot(1,2,2)
    fig = heart_failure_data[var].hist(bins=20)
    fig.set_ylabel('number of cases')
    fig.set_xlabel(var)
    
    plt.show()

Imbalanced Target Distribution

apparently certain ML algorithms such as trees tend to bias their predictions towards the majority answer. Lets see where our answer biases towards and how we can possible prevent a bias prediction algorithm. 

* "Oversampling is a technique which attempts to add random copies of the minority class to the dataset until the imbalance is eliminated". [[reference quote]](https://towardsdatascience.com/binary-classification-example-4190fcfe4a3c)
* "Undersampling is the opposite of oversampling as it entails removing majority class observations"[[reference quote]](https://towardsdatascience.com/binary-classification-example-4190fcfe4a3c) - disadvantage is you dont want to remove data
* "synthetic minority oversampling technique (SMOTE) uses the KNN algorithm to generate new observations to eliminate the imbalance." [[reference quote]](https://towardsdatascience.com/binary-classification-example-4190fcfe4a3c)

In [ ]:
heart_failure_data['DEATH_EVENT'].value_counts()/len(heart_failure_data)

# ## Step 4: Preprocessing Pipilenine


In [ ]:
heart_failure_data[discrete_features] = heart_failure_data[discrete_features].astype('O')
X = heart_failure_data[heart_failure_features]
y = heart_failure_data.DEATH_EVENT

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)



## Step 5: Create Model

(If I were to redo anything, it is this section. I did not properly understand everything I implemented)

What I think I should have:

y: DEATH_EVENT

X: age, anaemia, creatinine_phosphokinase, ejection_fraction, high_blood_pressure, platelets, smoking.

i think time can play a role in a way where it can weight certain sections. (e.g if all the people who died soon afer the checkup had high_blood_pressure than there is good chance that the next person will too. I just dont know how the model works and if it would consider something like that.)

THIS IS A BOOLEAN CLASSIFICATION PROBLEM, HOW DOES THIS AFFECT YOUR WORK?

In [ ]:
models = []
models.append(('log_reg', LogisticRegression(max_iter=10000, random_state=42)))
models.append(('rf_classifer', RandomForestClassifier(random_state=42)))
models.append(('bayes', GaussianNB()))
models.append(('gbc', GradientBoostingClassifier()))
base_model_train = []
base_model_test = []
for name, classifier in models:
    scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='recall')
    base_model_train.append(scores.mean().round(4))
    print(scores)
    print('{}: Avg CV recall using all features on training data: {}'.format(name, scores.mean().round(4)))
    
    classifier.fit(X_train, y_train)
    y_preds = classifier.predict(X_test)
    test_recall = recall_score(y_test, y_preds, average='binary')
    test_class = classification_report(y_test, y_preds)
    cnf_matrix = confusion_matrix(y_test, y_preds)
    base_model_test.append(test_recall.round(4))
    print('{}: Recall w/all features on test data {}:'.format(name, test_recall.round(4)))
    print(test_class)
    print(cnf_matrix)
    print('-------------------------------------------------------')

In [ ]:
sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
X_train_std_sm, y_train_sm = sm.fit_resample(X_train, y_train)
model1_train = []
model1_test = []
for name, classifier in models:
    pipeline = make_pipeline(classifier)
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
    model1_train.append(scores.mean().round(4))
    print(scores)
    print('{}: Avg CV Recall w/All Reatures: {}'.format(name, scores.mean().round(4)))
    
    classifier.fit(X_train_std_sm, y_train_sm)
    y_preds = classifier.predict(X_test)
    test_recall = recall_score(y_test, y_preds)
    test_class = classification_report(y_test, y_preds)
    cnf_matrix = confusion_matrix(y_test, y_preds)
    model1_test.append(test_recall.round(4))
    print('{}: Recall w/All Features on test data {}:'.format(name, test_recall.round(4)))
    print(test_class)
    print(cnf_matrix)
    print('-------------------------------------------------------')

In [ ]:
models = []
models.append(('log_reg', LogisticRegression(max_iter=10000, random_state=42)))
models.append(('rf_classifer', RandomForestClassifier(random_state=42)))
models.append(('bayes', GaussianNB()))
models.append(('gbc', GradientBoostingClassifier()))
model2_train = []
model2_test = []
for name, classifier in models:
    scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='recall')
    model2_train.append(scores.mean().round(3))
    print(scores)
    print('{}: Avg CV Recall on RF Features: {}'.format(name, scores.mean().round(3)))
    
    classifier.fit(X_train, y_train)
    y_preds = classifier.predict(X_test)
    test_recall = recall_score(y_test, y_preds, average='binary')
    test_class = classification_report(y_test, y_preds)
    cnf_matrix = confusion_matrix(y_test, y_preds)
    model2_test.append(test_recall.round(3))
    print('{}: Recall w/RF features on test data {}:'.format(name, test_recall.round(3)))
    print(test_class)
    print(cnf_matrix)
    print('-------------------------------------------------------')

In [ ]:

models = []
models.append(('log_reg', LogisticRegression(max_iter=10000, random_state=42)))
models.append(('rf_classifer', RandomForestClassifier(random_state=42)))
models.append(('bayes', GaussianNB()))
models.append(('gbc', GradientBoostingClassifier()))
sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
#skf = StratifiedKFold(n_splits=5)
model3_train = []
model3_test = []
for name, classifier in models:
    pipeline = make_pipeline(classifier)
    scores = cross_val_score(pipeline, X_train, y_train, scoring='recall')
    model3_train.append(scores.mean().round(3))
    print(scores)
    print('{}: Avg CV Recall w/RF Reatures: {}'.format(name, scores.mean().round(3)))
    
    classifier.fit(X_train, y_train)
    y_preds = classifier.predict(X_test)
    test_recall = recall_score(y_test, y_preds, average='binary')
    test_class = classification_report(y_test, y_preds)
    cnf_matrix = confusion_matrix(y_test, y_preds)
    model3_test.append(test_recall.round(3))
    print('{}: Recall w/RF on test data {}:'.format(name, test_recall.round(3)))
    print(test_class)
    print(cnf_matrix)
    print('-------------------------------------------------------')

In [ ]:
classifiers = ['Log_Regression', 'Random_Forest', 'Naive_Bayes', 'Gradient_Boosting_clf']
idx = ['All_Feat_Imbalance_Train', 'All_Feat_Imbalance_Test','All_Feat_Smote_Train',
'All_Feat_Smote_Test','RF_Imbalance_Train', 
'RF_Imbalance_Test','RF_Smote_Train','RF_Smote_Test']
combined_results = pd.DataFrame([base_model_train,base_model_test,
model1_train, model1_test, model2_train,
model2_test, model3_train, model3_test],
columns=classifiers, index=idx)
test_results = pd.DataFrame([base_model_test, model1_test, model2_test, model3_test], columns=classifiers, index=idx[1:8:2])
print(test_results)

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(data=test_results[['Log_Regression', 'Random_Forest',
                             'Naive_Bayes', 'Gradient_Boosting_clf']])
plt.xlabel('Iterations', fontsize=17, labelpad=15)             
plt.ylabel('Recall Scores', fontsize=17, labelpad=15)
plt.title('Classifier Recall Scores on Test Data',fontsize=25, pad=15)
plt.show()